# Quick start for fitting and using the DREAM model

### 1) Specify your settings here, including the full path to the `olfaction_prediction` folder (probably two levels above this file)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Where is your olfaction_prediction folder?
OP_PATH = '/path/to/olfaction-prediction'
OP_PATH = '/mnt/d/Dropbox (ASU)/science/olfaction-prediction'
OP_PATH = '/Users/rgerkin/Dropbox (ASU)/science/olfaction-prediction'

# Which feature sets do you want to use?
FEATURE_SETS = ['dragon', 'episuite', 'morgan', 'nspdk', 'gramian']

# Which datasets do you want to train on?
TRAIN_ON = ['training', 'leaderboard']

# Which datasets do you want to test on?
TEST_ON = ['testset']

# Which subjects do you want to build a model for (or for the mean or stdev across subjects)
SUBJECT = 'Mean' # Other options are 'StDev' or an integer subject number

# What dilution do you want to evaluate model quality on (other dilutions will still be used for training)
DILUTION = 'gold' # An integer (e.g. -3 for 1:1000) or the values 'high', 'low', or 'gold'

### 2) Load all libraries

In [3]:
# Built-in or external libraries
%matplotlib inline
import os
import sys
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas

In [4]:
# Adds olfaction-prediction directory from environment or user variable to the Python path
OP_PATH = os.environ.get('OP_PATH', OP_PATH)
sys.path.append(OP_PATH)

In [5]:
# Internal libraries
import opc_python
# For loading data into dataframes
from opc_python.utils import loading
# dream contains lots of important functions related to model building
# fit2 is specifically subchallenge 2 related (fitting mean and stdev) when the data is ready
# params contains functions for obtaining or looking up previously obtained hyperparameters
from opc_python.gerkin import dream, params, fit1, fit2

### 3) Load all data

In [6]:
# Get all descriptor names
descriptors = loading.get_descriptors(format=True)

# Get all CIDs from the DREAM dataset
all_CIDs = loading.get_CIDs([TRAIN_ON, TEST_ON])

# Get all CIDs and dilutions for training data
# CID + dilution completely defines the stimulus, and can be used for filtering the data
# May take a few seconds the first time it is run, then uses a pickled file for better performance
all_CID_dilutions = loading.get_CID_dilutions([TRAIN_ON, TEST_ON])
training_CID_dilutions = loading.get_CID_dilutions(TRAIN_ON)
testing_CID_dilutions = loading.get_CID_dilutions(TEST_ON)

In [7]:
# Get all molecular data, i.e. physicochemical features
molecular_data = loading.get_molecular_data(FEATURE_SETS, all_CIDs)

Dragon has 4869 features for 476 molecules.
Episuite has 62 features for 476 molecules.
Morgan has 2437 features for 476 molecules.
Nspdk has 6163 features for 476 molecules.
Gramian has 2437 features for 476 molecules.
There are now 15968 total features.


In [8]:
# Get all perceptual data, i.e. odor ratings
perceptual_data = loading.load_perceptual_data([TRAIN_ON, TEST_ON])

### 4) Build the matrix of predictors and the vectors of target data

In [9]:
# Create the X matrix from the molecular data in the training set.
# Several other arguments are available; see dream.py
X, metadata = dream.make_X(molecular_data, all_CID_dilutions)

The X matrix now has shape (952x8011) molecules by non-NaN good molecular descriptors


In [10]:
# Create the Y matrix from the perceptual data in the training set.
# Several other arguments are available; see dream.py
Y = dream.make_Y(perceptual_data)

In [11]:
print(("There are %d CID/dilution combinations for which we have molecular data but not "
       "perceptual data. ") % len(set(X.index).difference(Y.index)))

# Remove those CID/dilution combinations from the molecular data
X = X.loc[Y.index]

There are 42 CID/dilution combinations for which we have molecular data but not perceptual data. 


In [12]:
# Update the testing CID/dilutions combinations to reflect this
testing_CID_dilutions = list(set(testing_CID_dilutions).intersection(Y.index))

Note that the above number may be greater than 0 if some of the testset CID/dilution combinations were not present in the DREAM challenge data, for example molecules for which the highest tested concentration was 1/1000, so this concentration was used for all descriptors in the challenge, and thus the data for the lower concentration was never reported. 

### 5) Split into training and testing sets

In [13]:
X_train = X.loc[training_CID_dilutions]
X_test = X.loc[testing_CID_dilutions]
Y_train = Y.loc[training_CID_dilutions]
Y_test = Y.loc[testing_CID_dilutions]
assert X_train.shape[0] == Y_train.shape[0]
assert X_test.shape[0] == Y_test.shape[0]
print("We now have %d CID/dilution combinations for training and %d for testing" \
      % (X_train.shape[0], X_test.shape[0]))

We now have 814 CID/dilution combinations for training and 96 for testing


In [14]:
# Impute missing values for perceptual training data
Y_imp = dream.impute(Y_train, 'median')

### 6) Load previously computed hyperparameters for model fitting

In [15]:
# Load hyperparameters previously obtained via cross-validation
# use_et = use ExtraTrees instead of RandomForest
# regularize = balance between subject-specific fits and population fits (not used for mean prediction)
# use_mask = use masked data instead of imputed data for training
hp = params.get_hyperparams('Mean')
hp.head()

,use_et,max_features,max_depth,min_samples_leaf,regularize,use_mask
Intensity,True,None,None,1,0.8,False
Pleasantness,False,None,None,1,0.7,False
Bakery,False,None,6,1,0.9,False
Sweet,False,None,None,1,0.8,False
Fruit,False,None,15,1,0.8,False


In [16]:
# Determine translation parameters 
# (for translating mean predictions into standard deviation predictions)
trans_params = params.get_trans_params(Y, descriptors, plot=False)

### 7) Fit the models for each descriptor

In [17]:
# Increase n_estimators to get better fits; full model uses ~100.  
# Set std to True to also predict standard deviations
models = fit2.rfc_fit_models(X_train, Y_train, Y_imp, hp, n_estimators=25, std=False)

[-----------------------100%---------------------] 21 out of 21 complete (All descriptors' models have been fit)


### 8) Make predictions on the testset data for each model and descriptor

In [18]:
# Get predicted ratings using the models applied to the test molecular data, restricted to DILUTION
predicted = fit2.rfc_get_predictions(models, X_test, dilution=DILUTION)

### 9) Check predictions against testset data

In [19]:
# Get the observed data for the test molecules, restricted to DILUTION
observed = fit2.get_observed(Y_test, dilution=DILUTION)

In [20]:
# For each descriptor, print the Pearson correlation between the predicted and observed ratings
for descriptor in descriptors:
    p = predicted['mean'][descriptor]
    o = observed['mean'][descriptor]
    r, p = pearsonr(p, o)
    print('R = %.3g for %s' % (r, descriptor))

R = 0.704 for Intensity
R = 0.568 for Pleasantness
R = 0.522 for Bakery
R = 0.645 for Sweet
R = 0.589 for Fruit
R = 0.743 for Fish
R = 0.796 for Garlic
R = 0.648 for Spices
R = 0.186 for Cold
R = 0.57 for Sour
R = 0.581 for Burnt
R = 0.418 for Acid
R = 0.474 for Warm
R = 0.584 for Musky
R = 0.556 for Sweaty
R = 0.337 for Ammonia
R = 0.415 for Decayed
R = 0.303 for Wood
R = 0.499 for Grass
R = 0.661 for Flower
R = 0.141 for Chemical
